<a href="https://colab.research.google.com/github/nidheesh-p/AI-Learning/blob/master/the_need_for_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Need for RAG
Before we start coding, lets go over a few questions and get it clarified.

### 1. What is RAG?
Imagine you're writing an article about climate change, but instead of relying only on what you remember, you search online for recent studies and data to support your writing. RAG works the same way—it combines a powerful AI language model with a search system that retrieves relevant information from an external knowledge source, like a database or documents. This helps generate more accurate and informative responses.

### 2. Why is RAG important?
RAG is crucial because AI models, like ChatGPT, can sometimes "hallucinate" or provide outdated or incorrect information. By retrieving facts from trusted sources before generating responses, RAG ensures the answers are more reliable, up-to-date, and contextually relevant.

### 3. What’s the difference between RAG and a standard AI chatbot?
A standard chatbot relies only on pre-trained knowledge, which may be limited or outdated. RAG-enhanced chatbots, however, actively retrieve fresh, relevant information from external sources, ensuring better accuracy and up-to-date insights.

### 4. What are the key components of RAG?
RAG consists of two main parts:

- Retriever: Finds the most relevant documents or data from a knowledge base (e.g., a search engine or database).
- Generator: Uses the retrieved information to produce a coherent and accurate response.

### 5. Usecases in real-life
- Customer Support: AI chatbots retrieve knowledge base articles to provide better responses to customer queries.
- Healthcare: Doctors can get AI-assisted summaries of patient records and the latest medical research.
- Legal Services: Lawyers can search through legal documents and case studies to build stronger cases.

### Understanding the Limitation of the LLM

In [ ]:
# Importing the OpenAI library to interact with OpenAI's API services.
from openai import OpenAI

In [ ]:
import os  # Importing the os module to interact with environment variables
import getpass  # Importing getpass to securely input sensitive information

# Prompting the user to securely enter their OpenAI API key without displaying it on the screen
OPENAI_API_KEY = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [ ]:
# Creating an instance of the OpenAI client using the provided API key.
client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
# Defining the prompt to query the LLM
prompt = ''' What was uber's revenue in 2022? '''

In [ ]:
# Sending a request to the OpenAI API to generate a chat response
openai_response = client.chat.completions.create(
    model='gpt-3.5-turbo',  # Specifying the model to use;
    # Note: An older model chosen for testing purposes because the cutoff is 2021 whereas prompt is querying details about 2022
    messages=[{'role': 'user', 'content': prompt}]  # Creating a structured message for the AI model
    # use: what was uber's revnue...
)


In the above code, while creating a structured message for the AI model,  `role `defines the speaker (user input) and `content` contains the actual query stored in the `prompt` variable.

We are structuring the input this way because OpenAI's chat models require a specific format to understand and process conversations effectively. Assigning roles like 'user' helps the AI distinguish between different participants in the conversation, ensuring it provides relevant and context-aware responses.

In [ ]:
# Accessing the generated response from the AI model.
print(openai_response.choices[0].message.content)
# Note:'choices' contains multiple response options, we take the first one ([0]),
# 'message' holds the response details, and 'content' extracts the actual text generated by the AI.

As of now, it is not possible to provide the exact revenue for Uber in 2022 as it is still early in the year. Typically, companies release their revenue figures at the end of the fiscal year or quarterly. You may want to check Uber's financial reports or official announcements for the most recent revenue information.


### Interpretation:
Why is the LLM not able to answer the query?

`gpt-3.5-turbo` does not have access to data after 2021 because of its cutoff.

#### Do it yourself:
Try changing the model to `gpt-4o-mini` and observe how the output changes!

As you can see above, the LLM we used(gpt 3.5) doesn't have access to the latest data. Now as LLMs get updated, the training cut-off date may or may not have access to more information. However, it's always a good idea to understand how to improve the context of our prompt

### Making the LLM context-aware

Next Step: Let's check Uber's [financial report ](https://s23.q4cdn.com/407969754/files/doc_events/2024/May/06/2023-annual-report.pdf)

On Page 54, of the above document it states:

"Revenue was 37.3 billion, up 17% year-over-year. Mobility revenue increased 5.8 billion primarily attributable to an increase in
Mobility Gross Bookings of......"

In [ ]:
## Let's create the above context for the prompt
# Defining a context string with revenue details retrieved from an external source.
retrieved_context = '''Revenue was $37.3 billion, up 17% year-over-year. Mobility revenue increased $5.8 billion primarily attributable to an increase in
               Mobility Gross Bookings of 31% year-over-year.'''

In [ ]:
## Let's modify our prompt now
# Creating a prompt by embedding the retrieved context into a question for the AI model.

prompt = f"What was Uber's revenue in 2022? Check in {retrieved_context}"

# Note: The AI is being asked to analyze the given context and provide Uber's revenue for 2022

In [ ]:
print(prompt)

What was Uber's revenue in 2022? Check in Revenue was $37.3 billion, up 17% year-over-year. Mobility revenue increased $5.8 billion primarily attributable to an increase in
               Mobility Gross Bookings of 31% year-over-year.


In [ ]:
## Let's ask the LLM again
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': prompt}])

In [ ]:
# Accessing the generated response from the AI model.
openai_response.choices[0].message.content

"Therefore, Uber's revenue in 2022 was $37.3 billion."

### Interpretation:
How is the LLM able to answer the same question now?

The LLM can now answer the question accurately because the relevant financial data is explicitly provided in the `retrieved_context`, allowing the model to reference it directly instead of relying on its pre-trained knowledge.

As you saw in the example above, we

- **retrieved** the context from an external source
- **augmented** our prompt that passes to the LLM, and
- **generated** the response

This is Retrieval Augmented Generation in a nutshell!

### Basic RAG app architecture

In the previous example, we ***manually*** retrieved the context from the given file which for all purposes is impractical!

Therefore, we have to devise a strategy that enables us to:

- Take the query from the user
- Identify the documents from the external source that might be relevant for the query.
- Pass those documents' information as context to the LLM
- LLM then generates the final response


To do the above, we can follow a simple standard architecture as shown below (Image source - https://huyenchip.com/2024/07/25/genai-platform.html)

<center><img src="https://huyenchip.com/assets/pics/genai-platform/3-rag.png" width=500 height=400/></center>

In the subsequent demos today, we shall see how to do the same